## Intancia um objeto MinioSpark

In [1]:
from minio_spark import MinioSpark
datalake = MinioSpark()

## Lista todos os buckets do datalake

In [2]:
buckets = datalake.client.list_buckets()
print("Buckets in the root of the DataLake:")
for bucket in buckets:
    print(bucket.name)

Buckets in the root of the DataLake:
raw
stage


## Lista todos os arquivos do bucket "raw" que iniciam com "GSH_APAC"

In [4]:
raw_bucket = datalake.get_bucket('raw')
for obj in raw_bucket.list_objects(prefix="GSH_APAC"):
    print(obj.name)

GSH_APAC_1.zip
GSH_APAC_2.zip
GSH_APAC_3.zip
GSH_APAC_4.zip
GSH_APAC_5.zip


## Carrega os arquivos .zip em um dataframe spark.

O método `read_csv_from_zip` tem como objetivo ler arquivos CSV compactados em um arquivo ZIP. O processo ocorre em três etapas:

1. **Criação da pasta:**

Um objeto Minio é utilizado para criar uma pasta temporária com o nome do prefixo fornecido, no exemplo "GSH_APAC". Essa pasta servirá para armazenar os arquivos descompactados.

2. **Descompactação dos arquivos:**

Todos os arquivos dentro do arquivo ZIP serão descompactados e extraídos para a pasta criada na etapa anterior. Isso garante que os arquivos CSV fiquem acessíveis para leitura.

3. **Leitura dos arquivos CSV:**

Finalmente, o método lê cada um dos arquivos CSV descompactados na pasta "GSH_APAC". O conteúdo desses arquivos pode ser processado conforme as necessidades da aplicação.

In [7]:
df = datalake.read_csv_from_zip('raw', 'GSH_APAC', delimiter='|')
df.count()

3677282

## Lista os arquivos descompactados na pasta "GSH_APAC"

In [9]:
raw_bucket = datalake.get_bucket('raw')
for obj in raw_bucket.list_objects(prefix="GSH_APAC/"):
    print(obj.name)

GSH_APAC/GSH_APAC_1.txt
GSH_APAC/GSH_APAC_2.txt
GSH_APAC/GSH_APAC_3.txt
GSH_APAC/GSH_APAC_4.txt
GSH_APAC/GSH_APAC_5.txt


## Carrega o arquivo (objeto MinIO) GSH_APAC/GSH_APAC_1.txt em um datafame

In [7]:
df = datalake.read_csv_to_dataframe('raw', 'GSH_APAC/GSH_APAC_1.txt', delimiter='|')
df.count()

956914

In [17]:
df.select('APAC_ID', 'APCIDPRINC_CID', 'APCIDPRINC_DESCR_CID').show(truncate=False)

+-------+--------------+--------------------------------------------+
|APAC_ID|APCIDPRINC_CID|APCIDPRINC_DESCR_CID                        |
+-------+--------------+--------------------------------------------+
|4      |M810          |OSTEOPOROSE POS-MENOPAUSICA                 |
|5      |L700          |ACNE VULGAR                                 |
|6      |L700          |ACNE VULGAR                                 |
|7      |N180          |DOENC RENAL EM ESTADIO FINAL                |
|8      |E78           |DISTURBIOS METAB LIPOPROTEINAS E OUT LIPIDEM|
|9      |G300          |DOENC DE ALZHEIMER DE INICIO PRECOCE        |
|10     |F900          |DISTURBIOS DA ATIVIDADE E DA ATENCAO        |
|11     |E228          |OUTR HIPERFUNCOES DA HIPOFISE               |
|12     |L708          |OUTR FORM DE ACNE                           |
|13     |F200          |ESQUIZOFRENIA PARANOIDE                     |
|14     |L700          |ACNE VULGAR                                 |
|15     |M068       

In [11]:
df.printSchema()

root
 |-- APAC_ID: string (nullable = true)
 |-- APAC_ID_ANTERIOR: string (nullable = true)
 |-- APAC_PPI_MOVIMENTACAO: string (nullable = true)
 |-- APAC_USUARIO_AUTORIZADOR: string (nullable = true)
 |-- APAC_DATA_AUTORIZACAO: string (nullable = true)
 |-- APAC_DATA_FIM: string (nullable = true)
 |-- APAC_DATA_INICIO: date (nullable = true)
 |-- APAC_DATA_PREVISAO_AUTORIZACAO: integer (nullable = true)
 |-- APAC_DATA_REJEICAO: string (nullable = true)
 |-- APAC_DATA_SOLICITACAO: string (nullable = true)
 |-- APAC_DT_ENCERRAMENTO: string (nullable = true)
 |-- APAC_DT_FINALIZACAO_APAC: string (nullable = true)
 |-- APAC_DT_SENHA_TRIAGEM: string (nullable = true)
 |-- APAC_DV_APAC: string (nullable = true)
 |-- APAC_EXAMES_CLINICOS: string (nullable = true)
 |-- APAC_FLAG_ESTADO: integer (nullable = true)
 |-- APAC_FLAG_NUMERACAO: string (nullable = true)
 |-- APAC_FLAG_RECEITA_SUS: string (nullable = true)
 |-- APAC_ID_APAC_CANCELA_MOTIVO: string (nullable = true)
 |-- APAC_ID_APAC_MO